In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator


In [2]:
tf.__version__

'2.2.0'

### RUN THE CODE BELOW ONLY ONCE 

In [16]:
# load dataset, reshape and save to a new file
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
# define location of dataset
folder = 'dataset/train/'
photos, labels = list(), list()
# enumerate files in the directory
for file in listdir(folder):
	# determine class
	output = 0.0
	if file.startswith('b'):
		output = 1.0
	# load image
	photo = load_img(folder + file, target_size=(200, 200))
	# convert to numpy array
	photo = img_to_array(photo)
	# store
	photos.append(photo)
	labels.append(output)
# convert to a numpy arrays
photos = asarray(photos)
labels = asarray(labels)
print(photos.shape, labels.shape)
# save the reshaped photos
save('contaminated_vs_clean_photos.npy', photos)
save('contaminated_vs_clean_labels.npy', labels)

(96, 200, 200, 3) (96,)


In [17]:
# load and confirm the shape
from numpy import load
photos = load('contaminated_vs_clean_photos.npy')
labels = load('contaminated_vs_clean_labels.npy')
print(photos.shape, labels.shape)

(96, 200, 200, 3) (96,)


In [20]:
# create directories
import os
dataset_home = 'dataset_contaminated_vs_clean/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
	# create label subdirectories
	labeldirs = ['contaminated/', 'clean/']
	for labldir in labeldirs:
		newdir = dataset_home + subdir + labldir
		os.makedirs(newdir, exist_ok=True)

In [25]:
# seed random number generator
import random
import shutil
random.seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.25
# copy training dataset images into subdirectories
src_directory = 'dataset/train/'
for file in listdir(src_directory):
	src = src_directory + '/' + file
	dst_dir = 'train/'
	if random.random() < val_ratio:
		dst_dir = 'test/'
	if file.startswith('b'):
		dst = dataset_home + dst_dir + 'contaminated/'  + file
		shutil.copyfile(src, dst)
	elif file.startswith('n'):
		dst = dataset_home + dst_dir + 'clean/'  + file
		shutil.copyfile(src, dst)

### ---------------------TILL HERE--------------------

In [105]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

In [106]:
# define model
model = define_model()

ResourceExhaustedError: OOM when allocating tensor with shape[320000,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [100]:
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0)

In [101]:
# prepare iterators
train_it = datagen.flow_from_directory('dataset_contaminated_vs_clean/train',
	class_mode='binary', batch_size=64, target_size=(200, 200))
test_it = datagen.flow_from_directory('dataset_contaminated_vs_clean/test',
	class_mode='binary', batch_size=64, target_size=(200, 200))

Found 29 images belonging to 2 classes.
Found 12 images belonging to 2 classes.


In [102]:
# fit model
history = model.fit(train_it, steps_per_epoch=len(train_it),
	validation_data=test_it, validation_steps=len(test_it), epochs=20, verbose=0)

ResourceExhaustedError:  OOM when allocating tensor with shape[29,32,200,200] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_4/conv2d_3/Conv2D (defined at <ipython-input-80-3111f33efc95>:2) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_6353]

Function call stack:
train_function


In [ ]:
# save model
model.save('final_model.h5')

In [103]:
# evaluate model
_, acc = model.evaluate(test_it, steps=len(test_it), verbose=0)
print('> %.3f' % (acc * 100.0))

> 91.667


In [104]:
# make a prediction for a new image.
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(200, 200))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 200, 200, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img

# load an image and predict the class
def run_example():
	# load the image
    img = load_image('sample_3.jfif')
	# load model
    model = load_model('final_model.h5')
# predict the class
    result = model.predict(img)
    if result[0] == 1:
      prediction = 'contaminated'
    else:
      prediction = 'clean'
    print(prediction)

# entry point, run the example
run_example()

ResourceExhaustedError: OOM when allocating tensor with shape[320000,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add]